In [13]:
%%capture
!pip install open3d
!pip install torchinfo

In [38]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchinfo import summary
from torchvision import transforms
import tqdm

import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import open3d as o3d
import plotly.graph_objects as go


if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 2.2.1+cu121  Device: cpu


In [3]:
from google.colab import drive
drive.mount("/content/drive")

drive_path = "/content/drive/MyDrive/CV/MPI-FAUST"
training_path = drive_path + "/training"
test_path = drive_path + "/test"

Mounted at /content/drive


In [4]:
text_file = open(training_path+"/ground_truth_vertices/tr_gt_000.txt", "r")
lines = text_file.readlines()
print(len(lines))
vertices = []
for i in range(len(lines)):
    vertices.append(int(lines[i].rstrip('\n')))
text_file.close()

176387


In [ ]:
# Scan

cloud_mesh_scan = o3d.io.read_triangle_mesh(training_path+"/scans/tr_scan_000.ply")
np.asarray(cloud_mesh_scan.vertices)
cloud_mesh_scan.compute_vertex_normals()
cloud_mesh_scan.compute_triangle_normals()

TriangleMesh with 176387 points and 352169 triangles.

In [5]:
# Registration

cloud_mesh_reg = o3d.io.read_point_cloud(training_path+"/registrations/tr_reg_000.ply")

In [6]:
points_reg = np.asarray(cloud_mesh_reg.points)
colors_reg = np.asarray(cloud_mesh_reg.colors)
fig = go.Figure(
    data=[
        go.Scatter3d(
            x = points_reg[:,0], y=points_reg[:,1], z=points_reg[:,2],
            mode = 'markers',
            marker = dict(size=0.5, color=colors_reg)
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=True),
            yaxis=dict(visible=True),
            zaxis=dict(visible=True),
        )
    )
)

fig.show()

In [ ]:
vert = np.asarray(cloud_mesh_scan.vertices)
triang = np.asarray(cloud_mesh_scan.triangles)
colors_scan = np.asarray(cloud_mesh_reg.colors)
fig = go.Figure(
    data=[
        go.Mesh3d(
            x = vert[:,0], y=vert[:,1], z=vert[:,2],
            i = triang[:,0], j=triang[:,1], k=triang[:,2],
            facecolor=colors_scan,
            opacity=0.50
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=True),
            yaxis=dict(visible=True),
            zaxis=dict(visible=True),
        )
    )
)

fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [69]:
class FAUST_Dataset(Dataset):
    def __init__(self, data_path, transform=None):

        self.data_path = data_path
        self.clouds = self.importData(self.data_path)

    def __getitem__(self, index):

      return torch.Tensor.numpy(self.clouds[index])

    def __len__(self, index):
        # num samples, num points, coordinates
        return self.clouds.size()

    def importData(self, data_path, SAMPLING=True, SAMPLING_SIZE=3024):

        input_mat = []
        s = 0
        directory_path = data_path + "/scans/"
        for file in os.listdir(directory_path):
            if file.endswith(".ply"):
                file_mesh = o3d.io.read_triangle_mesh(directory_path+file)
                vert = np.asarray(file_mesh.vertices)
                if SAMPLING:
                    indices = np.random.choice(len(vert), size=SAMPLING_SIZE)
                    vert = vert[indices]
                s += len(vert)
                input_mat.append(vert)

        return torch.tensor(np.array(input_mat))

    def plotCloud(self, cloud):
        fig = go.Figure(
            data=[
            go.Scatter3d(
                x =cloud[:,0], y=cloud[:,1], z=cloud[:,2],
                mode = 'markers',
                marker = dict(size=0.5, color=[])
            )
        ],
        layout=dict(
            scene=dict(
                xaxis=dict(visible=True),
                yaxis=dict(visible=True),
                zaxis=dict(visible=True),
                )
            )
        )
        fig.show()


trsfrmt = transforms.Compose([
        transforms.ToTensor()
])


trainSet = FAUST_Dataset(training_path, transform=trsfrmt) # 100 files "training/scans/tr_scan_0xy.ply"
trainSet.plotCloud(trainSet.__getitem__(10))
print(trainSet.__getitem__(10))


#testSet = FAUST_Dataset(test_path) # 200 files "test/scans/test_scan_xyz.ply"

#train_loader = DataLoader(dataset=trainSet, batch_size=1, shuffle=True)
#test_loader = DataLoader(dataset=testSet, batch_size=1, shuffle=False)

[[ 0.02967095 -0.22766565 -0.23178579]
 [-0.20727572 -0.5453949  -0.26800099]
 [-0.12368963  0.35606465 -0.24187802]
 ...
 [-0.17577277 -0.29812115 -0.26341113]
 [ 0.07136334  0.5250755  -0.23027925]
 [ 0.16232859 -0.04340919 -0.28113407]]
